# Modelo RFM

El modelo RFM (Recency, Frequency, Monetary) es una técnica ampliamente utilizada en análisis de clientes para segmentarlos en función de tres factores:

- **Recency (R)**: Cuánto tiempo ha pasado desde la última compra del cliente.
- **Frequency (F)**: Cuántas compras ha realizado el cliente en un período de tiempo determinado.
- **Monetary (M)**: Cuánto ha gastado el cliente en total.

Para crear el modelo RFM, seguiré los siguientes pasos:

1. **Recency (R)**: Calculará el número de días desde la última compra (fecha de entrega) para cada cliente.
2. **Frequency (F)**: Contar el número de compras (órdenes) de cada cliente.
3. **Monetary (M)**: Sumar el GMV de todas las compras de cada cliente.

Además, conservaré las siguientes características para cada cliente:

- `ontime_num` promedio.
- El último valor de `is_graduated`.
- El último valor de `is_kam`.
- El último valor de `microzona`.
- `birthday`.
- `source_id`.


In [18]:
import pandas as pd
import datetime

In [19]:
# Cargando el archivo original
df_original = pd.read_csv('data/merged_vlp_otd_anonimized.csv', parse_dates=['deliver_date', 'birthday'])
df_original.shape

(269133, 22)

In [20]:
# Filtrando transacciones de los últimos 3 meses
end_date = df_original['deliver_date'].max()
start_date = end_date - pd.DateOffset(months=3)
df_last_3_months = df_original[df_original['deliver_date'].between(start_date + + datetime.timedelta(days=1), end_date)]

# Calculando la frecuencia promedio en días para cada cliente en los últimos 3 meses
df_frequency = df_last_3_months.groupby('customer_id').size().reset_index(name='purchase_count')
df_frequency.head()

,customer_id,purchase_count
0,1,3
1,2,9
2,3,17
3,4,13
4,5,19


In [4]:
all_customers = df_original['customer_id'].unique()
missing_customers = set(all_customers) - set(df_frequency['customer_id'])
missing_data = pd.DataFrame({'customer_id': list(missing_customers), 'purchase_count': 0})
purchase_frequency = pd.concat([df_frequency, missing_data], ignore_index=True)
purchase_frequency.rename(columns={'purchase_count':'frequency'})
purchase_frequency.shape

(8287, 2)

In [5]:
# Calculando Recency, Monetary y otras métricas como antes
rfm_corrected = df_original.groupby('customer_id').agg({
    'deliver_date': lambda x: (df_original['deliver_date'].max() - x.max()).days,  # Recency
    'gmv_on_3': 'mean',  # Monetary
    'ontime_num': 'mean',  # ontime_num promedio
    'is_graduated': 'last',  # último valor de is_graduated
    'is_kam': 'last',  # último valor de is_kam
    'microzone_source_id': 'last',  
    'birthday': 'first',
    'order_close_date':'last' #Last order
}).reset_index()

# Unimos la frecuencia promedio en días calculada a las métricas RFM
rfm_corrected = rfm_corrected.merge(purchase_frequency, on='customer_id', how='left')

# Renombrando las columnas
rfm_corrected.columns = ['customer_id', 'recency', 'monetary', 'ontime_avg', 'is_graduated_last', 'is_kam_last', 'microzona_last', 'birthday','last_order', 'frequency']
rfm_corrected.head()#Por cliente unico

,customer_id,recency,monetary,ontime_avg,is_graduated_last,is_kam_last,microzona_last,birthday,last_order,frequency
0,1,16,42.241430,1.00000,True,False,149.0,2021-09-17,2023-03-30,3
1,2,1,84.172771,1.00000,True,False,121.0,2021-04-09,2023-05-15,9
2,3,1,63.493172,0.93750,True,False,120.0,2021-01-08,2023-06-27,17
3,4,5,77.517644,1.00000,False,False,635.0,2022-03-31,2023-08-04,13
4,5,2,157.325215,0.97619,False,False,347.0,2022-10-07,2023-08-24,20


In [6]:
# Definiendo funciones para asignar scores basados en percentiles
def r_score(x, percentile_dict):
    """Asignar score para Recency."""
    if x <= percentile_dict[0.2]:
        return 5
    elif x <= percentile_dict[0.4]:
        return 4
    elif x <= percentile_dict[0.6]:
        return 3
    elif x <= percentile_dict[0.8]:
        return 2
    else:
        return 1

def fm_score(x, percentile_dict):
    """Asignar score para Frequency y Monetary."""
    if x <= percentile_dict[0.2]:
        return 1
    elif x <= percentile_dict[0.4]:
        return 2
    elif x <= percentile_dict[0.6]:
        return 3
    elif x <= percentile_dict[0.8]:
        return 4
    else:
        return 5

# Calcular percentiles para Recency, Frequency, y Monetary
percentiles = rfm_corrected[['recency', 'frequency', 'monetary']].quantile(q=[0.2, 0.4, 0.6, 0.8]).to_dict()

# Asignar scores R, F, y M a cada cliente
rfm_corrected['R'] = rfm_corrected['recency'].apply(r_score, args=(percentiles['recency'],))
rfm_corrected['F'] = rfm_corrected['frequency'].apply(fm_score, args=(percentiles['frequency'],))
rfm_corrected['M'] = rfm_corrected['monetary'].apply(fm_score, args=(percentiles['monetary'],))

# Calcular el score RFM total (suma de R, F, y M)
rfm_corrected['RFM_Score'] = rfm_corrected['R'] + rfm_corrected['F'] + rfm_corrected['M']

rfm_corrected[['customer_id', 'R', 'F', 'M', 'RFM_Score']].head()


,customer_id,R,F,M,RFM_Score
0,1,3,2,2,7
1,2,5,3,4,12
2,3,5,4,3,12
3,4,4,4,4,12
4,5,4,5,5,14


In [7]:
# Uniendo los scores RFM al DataFrame original
df_enriched = df_original.merge(rfm_corrected[['customer_id', 'R', 'F', 'M', 'RFM_Score']], left_on='source_id', right_on='customer_id', how='left')


df_enriched.shape


(269133, 27)

# Segmentación de Clientes basada en Scores RFM

## Descripción de Segmentos

### 1. Clientes VIP
- **Descripción**: Estos son clientes que han comprado recientemente, compran con frecuencia y gastan mucho.
- **Características**: Alto en Recency, Frequency y Monetary.

### 2. Clientes Leales
- **Descripción**: Estos clientes compran con frecuencia y han gastado una cantidad decente, pero no han comprado tan recientemente como los VIP.
- **Características**: Medio-Alto en Recency, Alto en Frequency y Monetary.

### 3. Clientes en Riesgo
- **Descripción**: Estos clientes solían comprar con frecuencia y gastar, pero ha pasado algún tiempo desde su última compra.
- **Características**: Bajo en Recency, Medio en Frequency y Monetary.

### 4. Clientes Churneados/Zombies
- **Descripción**: Estos son clientes que no han comprado en mucho tiempo, pero en algún momento tuvieron alguna frecuencia de compra y gasto.
- **Características**: Muy bajo en Recency, Bajo en Frequency y Monetary.



## Descubrimientos de los Segmentos

1. **Clientes VIP**: Representan 2.048 clientes. Tienen una recencia baja, una alta frecuencia y un alto valor monetario. 
2. **Clientes Leales**: Representan 1.989 clientes. Su recencia es un poco más alta, pero aún tienen una buena frecuencia y valor monetario.
3. **Clientes en Riesgo**: Representan 2.126 clientes. Tienen una recencia y frecuencia moderadas y un valor monetario intermedio.
4. **Clientes Churneados/Zombies**: Representan 2.124 clientes. No han comprado en mucho tiempo, pero en algún momento tuvieron frecuencia y gasto.

Estos segmentos nos permiten abordar estrategias específicas para cada grupo y entender mejor las características y necesidades de cada segmento.


#### Etiquetando clientes churneados

In [8]:
# Calcular el stock de seguridad basado en la frecuencia de compra
rfm_corrected['stock_seguridad'] = rfm_corrected['frequency'] * 0.25

# Establecer el umbral de churn como la `recency` más el `stock_seguridad`
rfm_corrected['umbral_churn'] = rfm_corrected['frequency'] + rfm_corrected['stock_seguridad']

# Identificar clientes churneados
# Aquí simplemente estamos marcando como churneado a aquellos cuya recency es mayor que el umbral.
rfm_corrected['churned'] = (rfm_corrected['recency'] > rfm_corrected['umbral_churn'])




In [9]:
rfm_corrected.churned.value_counts()

True     4352
False    3935
Name: churned, dtype: int64

In [10]:
# Ahora, segmenta a los clientes
bins = pd.qcut(rfm_corrected[~rfm_corrected['churned']]['RFM_Score'], q=3, labels=['C', 'B', 'A'])  
rfm_corrected.loc[~rfm_corrected['churned'], 'Segment'] = bins

# Convertir la columna 'Segment' a tipo string (no categórico)
rfm_corrected['Segment'] = rfm_corrected['Segment'].astype(str)

# Para clientes churneados, asignarles la categoría 'D'
rfm_corrected.loc[rfm_corrected['churned'], 'Segment'] = 'D'

# Cambiando los nombres de los segmentos para que sean más descriptivos
segment_mapping = {
    'A': 'Clientes VIP',
    'B': 'Clientes Leales',
    'C': 'Clientes en Riesgo/Zombies',
    'D': 'Clientes Churneados'
}
rfm_corrected['Segment'] = rfm_corrected['Segment'].map(segment_mapping)

# Mostrando el conteo de clientes en cada nuevo segmento
new_segment_counts = rfm_corrected['Segment'].value_counts().sort_index()
new_segment_counts


Clientes Churneados           4352
Clientes Leales               1461
Clientes VIP                   892
Clientes en Riesgo/Zombies    1582
Name: Segment, dtype: int64

## Análisis de Life Time Value (LTV), GMV promedio y frecuencia promedio de compra.

- LTV: Representa el valor neto proyectado de un cliente a lo largo de toda su relación con el negocio. Se calcula multiplicando el GMV promedio por la frecuencia promedio de compra y por el promedio de tiempo que un cliente sigue siendo cliente.
- GMV promedio: Es el ingreso bruto promedio que un cliente genera.
- Frecuencia promedio de compra: Es el número promedio de veces que un cliente hace una compra en un período determinado.

Hemos asignado scores RFM a cada cliente:

*R*: Score para Recency. Un valor más alto indica una recencia mayor (es decir, el cliente ha hecho una compra más recientemente).  
*F*: Score para Frequency. Un valor más alto indica una mayor frecuencia de compras en los ultimos 3 meses. 
*M*: Score para Monetary. Un valor más alto indica un gasto promedio más alto por parte del cliente  
*RFM_Score*: Es la suma total de los scores R, F y M.  
Con estos scores, podemos comenzar a segmentar a los clientes en diferentes grupos y responder preguntas de negocio relevantes.  

Algunas preguntas de negocio que podríamos abordar incluyen:

In [11]:
# Calcular la duración promedio de un cliente usando rfm_corrected
rfm_corrected['customer_duration'] = (pd.to_timedelta(rfm_corrected['last_order'] - pd.to_timedelta(rfm_corrected['recency'], unit='D'))) #arreglar

# Calcular LTV (Life Time Value) para cada cliente
# LTV se calcula como GMV promedio multiplicado por frecuencia promedio de compra
rfm_corrected['LTV'] = rfm_corrected['monetary'] * rfm_corrected['frequency']

# Calculando valores promedio a nivel general
avg_LTV = rfm_corrected['LTV'].mean()
avg_GMV = rfm_corrected['monetary'].mean()
avg_frequency = rfm_corrected['frequency'].mean()
avg_customer_duration = rfm_corrected['customer_duration'].mean()

avg_LTV, avg_GMV, avg_frequency, avg_customer_duration

/home/andres/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimelike.py:1187: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


TypeError: unsupported operand type(s) for +: 'Timedelta' and 'str'

1. Life Time Value (LTV) Promedio: El LTV promedio es de aproximadamente $1171. Esto significa que, en promedio, se espera que un cliente aporte $1171 en valor a lo largo de su tiempo como cliente.
2. GMV Promedio: El valor promedio de GMV por compra es de aproximadamente $84.
3. Frecuencia Promedio de Compra: En promedio, un cliente realiza aproximadamente 11.5 compras.
4. Duración Promedio del Cliente: La duración promedio de un cliente (tiempo entre el registro y la última orden) es de aproximadamente 175.36 días.

# STORAGE IN CSV

In [13]:
rfm_corrected

,customer_id,recency,monetary,ontime_avg,is_graduated_last,is_kam_last,microzona_last,birthday,last_order,frequency,R,F,M,RFM_Score,stock_seguridad,umbral_churn,churned,Segment
0,1,16,42.241430,1.00000,True,False,149.0,2021-09-17,2023-03-30,3,3,2,2,7,0.75,3.75,True,Clientes Churneados
1,2,1,84.172771,1.00000,True,False,121.0,2021-04-09,2023-05-15,9,5,3,4,12,2.25,11.25,False,Clientes Leales
2,3,1,63.493172,0.93750,True,False,120.0,2021-01-08,2023-06-27,17,5,4,3,12,4.25,21.25,False,Clientes Leales
3,4,5,77.517644,1.00000,False,False,635.0,2022-03-31,2023-08-04,13,4,4,4,12,3.25,16.25,False,Clientes Leales
4,5,2,157.325215,0.97619,False,False,347.0,2022-10-07,2023-08-24,20,4,5,5,14,5.00,25.00,False,Clientes VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8282,8283,169,22.749470,1.00000,False,False,150.0,2021-06-22,2023-03-15,0,1,1,1,3,0.00,0.00,True,Clientes Churneados
8283,8284,66,22.926310,1.00000,False,False,150.0,2022-09-10,2023-06-26,1,2,2,1,5,0.25,1.25,True,Clientes Churneados
8284,8285,36,48.210520,1.00000,False,False,1188.0,2022-09-23,2023-07-26,1,2,2,2,6,0.25,1.25,True,Clientes Churneados
8285,8286,239,20.932630,1.00000,True,False,1235.0,2021-08-20,2023-01-04,0,1,1,1,3,0.00,0.00,True,Clientes Churneados


In [ ]:
rfm_corrected.to_csv('data/rfm.csv')